# Containers from the standard template library

There are many ways to represent "collections" of objects in C++ : names of people, all prime numbers up to an integer, tables of merchandise with the associated prices ... The C++ standard library already gives us containers to cover a large number of use cases. 

All containers in C++ have a compile time fixed element type. The type of the elements is uniform through the whole the container. That type is highly customizable, but it can not vary from element to element, and it can not be changed after creating the container. This is because the element type becomes part of the name of the container-type as you will see.

## `std::vector`

The most frequently used container is `std::vector` representing a sequence of values stored consecutively in memory.
To use it, we need to include the header `vector`, i.e., write `#include <vector>` at the top somewhere. After that, we can create variables of the `vector` type.

In [ ]:
#pragma cling add_include_path("/p/project/training2312/local/include")

In [ ]:
#include <iostream>
#include <vector>
// ...
std::vector V{1, 2, 3, 4, 5, 6, 7, 8, 9};

The elements of a vector are accessed using the indexing operator `[]`. The first element has index 0, the second has index 1 etc.

In [ ]:
std::cout << V[0] << "\n";

The size of the vector is available using the member function `size()`.

In [ ]:
std::cout << "There are " << V.size() << " elements in the vector\n";

## Exercise:
You know (i) how to access an element. (ii) how many elements there are. (iii) how to write a for loop. 

Write a `for` loop to print all the elements of the vector.

For a non-constant `std::vector` object, element access using the indexing operator `[]` produces an L-value reference. It is possible to use, for example, `V[1]` on the left hand side in an assignment expression, although `V[1]` is not a name, but the result of an expression. The class `vector` is designed in such a way that the indexing produces an L-value reference.

In [ ]:
for (auto i = 0UL; i < V.size(); ++i) {
    V[i] = i * i * i;
}

A very common pattern with vectors, or for that matter with any C++ container, is to perform an action for each element. This is perfectly suited for the "range based for loop".

In [ ]:
for (auto&& element : V) {
    std::cout << element << "\n";
}

The range based for loop can be used when, what we want to do with the elements depends on the elements but does not depend on the index explicitly. Like in this example. Using `auto&&` to declare the loop variable, so that for each loop iteration, we create and work with a reference to an existing element in the vector rather than creating a copy. If we really need a copy, one can simply replace the `auto&&` with an `auto`.

Above, when we created the vector with `std::vector V{1,2,3,4,5,6,7,8,9}`, we made a `vector` of `int` elements of size 9, storing the values 1 through 9. The type of the elements stored in a vector variable, like `V` here, is fixed at compile time. In this case it is `int` because we initialized it with a sequence of `int` variables. If the sequence of values in the initializer does not have a uniform type, we have an error!

In [ ]:
// std::vector notV{1, 2.0, 3}; // If you uncomment this, you may have to restart the kernel

Once initialized like that, `V` can only store `int`s. Before C++17, one had to explicitly state the type of the elements, something like this:

In [ ]:
std::vector<int> precxx17_V{1, 2, 3, 4, 5, 6, 7, 8, 9};

That syntax is still valid, and is still useful in C++20, e.g., when you want to create a vector of integers without filling it with anything.

The further down the memory lane you decide to go, the uglier the syntax you have to work with:
```c++
std::vector<int> precxx11_V(9);
for (int i=0; i<9; ++i) precxx11_V[i] = i;
```
Let's stay in the present.

The angular brackets in typenames like `vector` contain the so called **template arguments**. In this instance, it specifies what type of element our vector is going to contain. Even when we do not explicitly specify the element type, like for the vecotr `V` above, it is still there. The compiler is doing some extra work and filling out some text for us: it looks at the initializer list, deduces the type of the elements, augments our attempt to initialize `std::vector` by replacing that with `std::vector<T>` with `T` being the inferred type of the elements. You can see that it has created vectors of exactly the same type for `V` and `precxx17_V` above: 

In [ ]:
typeof(V);
typeof(precxx17_V);

The `std::allocator<int>` you see in the output above is a utility `std::vector` uses to manage dynamic memory. The important point to remember is that the following are equivalent.
    
```c++
std::vector A{1, 2, 3, 4, 5};
std::vector<int> B{1, 2, 3, 4, 5};
```
The true name is in both cases, `std::vector<int, std::allocator<int> >`.
    
The compiler just does a bit more for us in the more recent language versions. The compiled binary code is identical. Consider the two following declarations: 

In [ ]:
std::vector A{1, 2, 3};
std::vector B{1.0, 2.0, 3.0};
typeof(A);
typeof(B);

Superficially, we specified only `std::vector` as the types for both `A` and `B`. But there is no such type as just plain `std::vector`. `std::vector` is what is called a class template. We should be writing the full name of the type as printed above. That could get tiring very quickly! That's why the compiler helps us. The mechanism by which the compiler infers the type of the elements of a vector here is called **Class Template Argument Deduction**, or CTAD. All standard library containers are CTAD enabled. Later in the course, you will learn how to teach the compiler how to do CTAD for your own hand written classes.

CTAD can not work unless we have a non-empty initializer list. To create empty vectors, we have to explicitly write what type of elements a vector will contain. That can not be left to be determined at program execution time.

**Take away:** there is always a fixed element type, even when you don't write it explicitly, and it is burried in the true name of the vector (or any other standard container).

Although the type of the elements is a compile time constant, the size of a `vector` is not. They can grow at program execution time. In the following example, we create a vector of integers, and fill it with with new values in a loop as long as the new value is less than 1000. 

In [ ]:
std::vector<double> nums; // empty vector, but the element type is double
auto curr = 1.;
while (curr < 1000.) {
    nums.push_back(curr);
    curr *= 1.1;
}
std::cout << "Content of the nums vector ...\n";
for (auto&& m : nums) std::cout << m << "\n";


Above, we iterated over the final `vector` using a range based for loop. We can do it in two other ways. One is of course the plain `for` loop over valid indexes. The other is using the so called "iterators". Iterators are usually defined for the container types. You can imagine them to be like little arrows which can point to one element along the sequence at any time. If you increment (decrement) the iterator, it moves to point to the next(previous) element. To see the element an iterator points to, you "dereference" an iterator. Containers usually have `begin()` and `end()` defined, so that `container.begin()` gives us an iterator pointing to the first element in the sequence, and `container.end()` gives us an iterator pointing to a non-existing element one past the last element in the sequence. They can be used like this:

In [ ]:
for (auto it = nums.begin(); it != nums.end(); ++it) {
    std::cout << *it << "\n";
}

Containers define their own iterator types, and it is possible to iterate over the elements using the iterators. `std::vector` is special in the sense that it also gives us indexed access to elements, so that a plain `for` loop is possible.

```c++
    for (auto i = 0UL; i < nums.size(); ++i)
        std::cout << nums[i] << "\n";
```

We saw how to insert elements at the back of a vector. It is also possible to insert anywhere in the middle. To insert into a vector, we need an iterator to mark a position to the right of the insertion point. Say, we want to insert a new element $-1$ into our first vector `V`, before the first element. We can get an iterator to the first element using the function `V.begin()`. The insertion will look like this:

In [ ]:
V.insert(V.begin(), -1);

In [ ]:
// To verify...
for (auto&& elem: V) std::cout << elem << "\n";

How about inserting $100$ before the 5th element ? The fifth element is reached by advancing the iterator four times from the beginning. So, ... 

In [ ]:
auto pos = V.begin();
std::advance(pos, 4);
V.insert(pos, 100);

In [ ]:
// To verify...
for (auto&& elem: V) std::cout << elem << "\n";

## `std::array`

`std::array` is the proper way to make **fixed length arrays** in C++. In many ways it is like `std::vector`, but it does not have any operations which can change its size. No `push_back`, no insertions or deletions. Not just that, its size must be known to the compiler. This is enforced by requiring that the size is a template parameter, i.e., goes inside the angular brackets. Anything that goes into those angular brackets must be known at compilation time. This is how we create arrays in C++:

In [ ]:
#include <array>
std::array<double, 10> masses; // presumably to store masses of 10 entities.

If we have an explicit initializer list, the CTAD mechanism discussed above can infer both the type and the number of elements in an array. So, in real life, we can write the above as:

In [ ]:
std::array masses{0.1, 0.2, 0.1, 0.1, 0.1, 0.8, 0.1, 0.2, 0.1, 0.1};

Like a vector, you can access elements using the `[]` indexing operator, and the size using the `size()` member function. The 3 ways of iterating over a vector described above work for `std::array` as well.

## `std::list`
Not all containers allow index access. For instance, `std::list` is a doubly linked linked-list type. Let's create a linked list and a vector, fill them both with the same elements, and then compare the locations in memory of the elements in both cases:

In [ ]:
#include <list>
#include <vector>

std::vector<double> vd;
std::list<double> ld;
for (auto x = 0.0; x < 1.0; x += 0.1) {
    vd.push_back(x);
    ld.push_back(x);
}
std::cout << "Elements and their memory locations for vector: \n";
for (auto&& element : vd) std::cout << element << "\t" << (size_t)(&element) << "\n";
std::cout << "Elements and their memory locations for list: \n";
for (auto&& element : ld) std::cout << element << "\t" << (size_t)(&element) << "\n";

You see that the elements of the vector are placed very regularly, in memory locations separated by 8 bytes, which is the size of a double. This means, the elements are stored in adjacent locations in memory with no gaps. In the case of the list, the elements are stored scattered in different areas of memory. The design of the list is very very different than the vector, although the standard library is written in such a way that code can be written in a generic way so as to work for both. Ask in the class for an in-depth discussion of what linked lists are, if you are curious. For now, just note that providing an indexing operator `[]` for a list would be possible, but very slow.

For most part, you can pretend that `std::list<T>` does not exist and use `std::vector<T>` for all your needs for sequencial containers. For small arrays whose size does not change very often, e.g., an array storing the names of the week days, the number of neighbour sites in a lattice etc., one could use `std::array`, to emphasize the fact that the number of elements in those arrays is not going to change.



## Associative containers `std::set`, `std::map`, `std::unordered_set`, `std::unordered_map`

The containers `std::set` and `std::unordered_set` store one instance of each element. For instance, imagine that you are trying to generate a list of all the people who logged in to a computer during a day, irrespective of how often they logged in. You start with a vector or list or each username from the log. Just put all of them in an `std::set`. It will ignore a new insertion if that element is already there, and add it if it isn't.

In [ ]:
#include <set>
std::vector<std::string> allusernames{"gandalf", "cirdan", "galadriel", "gandalf", "arwen", "gandalf"};
std::set<std::string> uniquenames;
for (auto&& elem : allusernames) uniquenames.insert(elem);
for (auto&& elem: uniquenames) std::cout << elem << "\n";

Notice that not only does `std::set` keep only unique entries, it sorts them too. `std::set` manages to keep track of unique elements by always maintaining a sorted sequence of elements. `std::unordered_set` is similar in that it also keeps only unique elements, but (shockingly!) it does not keep them in order! In the cell below, repeat the experiment above using `std::unordered_set` rather than `std::set`!

In [ ]:
// Repeat the above, but with std::unordered_set instead of std::set
#include <unordered_set>
std::vector<std::string> allusernames{"gandalf", "cirdan", "galadriel", "gandalf", "arwen", "gandalf"};
std::unordered_set<std::string> uniquenames;
for (auto&& elem : allusernames) uniquenames.insert(elem);
for (auto&& elem: uniquenames) std::cout << elem << "\n";

The containers `std::map` and `std::unordered_map` are key -> value mappings. For instance, if we want to keep track of scores in a multi-player game. Conceptually, we will have something like

| Team       | Points |
|-----------:|-------:|
| Griffindor | 203 |
| Hufflepuff | 212 |
| Ravenclaw  | 350 |
| Slytherin  | 363 |

This is a mapping from strings to integers. If we had a vector where the indexes were strings instead of integral numbers, we could write something like `v["Griffindor"] += 160`. That is the role of the containers `std::map` and `std::unordered_map`.

```c++
std::map<std::string, int> score;
score["Griffindor"] = 203;
score["Hufflepuff"] = 212;
score["Ravenclaw"] = 350;
score["Slytherin"] = 363;
```
Let's try that ...

In [ ]:
#include <map>
#include <iostream>
std::map<std::string, int> score;
score["Griffindor"] = 203;
score["Hufflepuff"] = 212;
score["Ravenclaw"] = 350;
score["Slytherin"] = 363;

for (auto&& [key, value] : score) {
    std::cout << key << "\t:\t" << value << "\n";
}

Notice that we created the `map` empty, and simply started assigning to particular keys. `map` is designed in such a way that when you access a key that was not previously present, you insert that key with the default value of the value type (0 for numeric types). We don't need to resize it beforehand or use special append operations like `push_back` for `vector`. This is convenient, but that comes at a price: if you now try to give Griffindor 160 more points and misspell ...

In [ ]:
score["griffindor"] += 160;
for (auto&& [key, value] : score) {
    std::cout << key << "\t:\t" << value << "\n";
}

`map` simply assumed there is a new key called "griffindor" and created a new entry for it. Sometimes, we want to be strict about whether or not keys exist and only modify content rather than inserting a new element. You can do that by using the `at` function rather than the `[]` indexing operator in the lines where you want to prevent insertion...

In [ ]:
#include <map>
std::map<std::string, int> score{{"Griffindor", 203}, {"Hufflepuff", 212}, {"Ravenclaw", 350}, {"Slytherin", 363}};

score.at("Griffindor") += 160; // This is OK
// score.at("griffindor") += 160; // This will throw an exception
for (auto&& [key, value] : score) {
    std::cout << key << "\t:\t" << value << "\n";
}

As with the `std::set`, `std::map` sorts its contents by the keys. Notice how we initialized the `map` with some key value pairs above. One should think of containers like `std::map` as a sequence of (key, value) pairs, and that is how we initialized above. In our range based for loop, we also get, for each position along the sequence a pair as the current element. The pair is represented by `std::pair`, and its parts are actually called `first` and `second`. Before C++17, one had to write the range for loop above a bit differently:

```c++
for (auto&& entry : score) {
    std::cout << entry.first << "\t:\t" << entry.second << "\n";
}
```

The syntax is still valid, but the newer syntax `auto [key, value] : score` allows us to choose what we want to call the components, so that we can attach better names which suit our context.

For a small map like the above, the entire content is easily reviewed. If on the other hand, we had a map (or any of the other associative containers)  with 10000 entries, and we want to check if it contains a particular key, we can do that as follows:

```c++
score.contains("Ravenclaw"); // This will give you true or false
```

Unfortunately, we can not try that in this interpreter, as the `contains` function was only introduced in C++20.

Before C++20, we used to check for the existence of a particular key as follows. It makes if you think about it, but the new `contains` function is a more direct expression of intent.

In [ ]:
score.find("Ravenclaw") != score.end()

As you can imagine, `std::unordered_map` can also be used in place of the `map` in the above, except that it will not keep its contents sorted. The unordered map is a type of container commonly known as hash tables, whereas the `map` is based on a B-tree. The python equivalent is a dictionary.
